# Golden Standard Parity Validation -- LIU BTP8 Integral Coil

## Purpose

This notebook validates that our Python analysis pipeline produces **identical results** to the legacy C++ analyzer (ffmm framework) when processing the same raw measurement data.

**Dataset:** LIU BTP8, **Integral** coil, session `20190717_161332`
**Compensation scheme:** A (absolute) / A-B-C+D (compensated)

**Success criterion**: All harmonics must match to within numerical precision (ideally < 1e-9 relative error).

## Theory Background

### Rotating Coil Measurement Principle

A rotating coil measures magnetic field harmonics by rotating a coil inside a magnet aperture. The induced voltage is integrated to obtain flux, which is then Fourier-transformed to extract harmonics.

### Key Formulas (from ffmm C++ and Bottura PDF)

1. **Drift Correction** (legacy C++ style):
   ```
   flux = cumsum(df - mean(df)) - mean(cumsum(df))
   ```
   This removes DC offset from the incremental signal and centers the integrated flux.

2. **FFT Normalization**:
   ```
   f_n = 2 * FFT(flux)[n] / N_samples
   ```
   The factor of 2 accounts for the two-sided FFT spectrum.

3. **Kn Calibration** (coil sensitivity):
   ```
   C_n = f_n / conj(kn) * Rref^(n-1)
   ```
   Where `kn` are complex calibration coefficients from the measurement head geometry.

4. **Rotation** (phase alignment to main field):
   ```
   C_n_rotated = C_n * exp(-i * (n-m) * angle_m)
   ```
   Where `m` is the magnet order (2 for quadrupole) and `angle_m = arg(C_m)`.

5. **Normalization** (to relative units):
   ```
   c_n = C_n / C_m * 10000
   ```
   Where `C_m` is the main-field harmonic. Result in "units" (1 unit = 10^-4 of main field).

### Harmonic Merge Strategy

- **Absolute channel (ABS)**: Single coil A -- used for main field (n <= m)
- **Compensated channel (CMP)**: Bucking combination A-B-C+D -- used for errors (n > m)

For a quadrupole (m=2): Use ABS for n=1,2 and CMP for n=3,4,...,15

---
## 1. Configuration

Edit this cell to point to your golden standard dataset.

In [ ]:
from pathlib import Path

# =============================================================================
# DATASET CONFIGURATION
# =============================================================================

# Integral coil dataset (LIU BTP8 quadrupole, session 20190717_161332)
DATASET_FOLDER = Path("../../golden_standards/golden_standard_01_LIU_BTP8/Integral/20190717_161332_LIU")

# Kn calibration file -- R45_PCB_N1: A (absolute), A-B-C+D (compensated)
# This is the correct Kn for this dataset (confirmed by Parameters.txt "shaft: R45_PCB_N1"
# and by parity matching: B2 matches to 0.5 ppm with this file).
KN_FILE = Path("../../golden_standards/golden_standard_01_LIU_BTP8/COIL_PCB/Kn_R45_PCB_N1_0001_A_ABCD.txt")

# =============================================================================
# MAGNET PARAMETERS  (from BTP8_20190717_161332_Parameters.txt)
# =============================================================================

MAGNET_ORDER = 2       # Quadrupole
R_REF_M = 0.059        # Reference radius [m] (59 mm)
L_COIL_M = 1.32209     # Coil length [m]
SHAFT_SPEED_RPM = 60   # Rotation speed (absolute value)

# =============================================================================
# CRITICAL: Samples per turn
# =============================================================================
# BTP8 format: 512 samples per revolution.
# Using the wrong value causes ~60x magnitude errors!
SAMPLES_PER_TURN = 512

# =============================================================================
# ANALYSIS OPTIONS
# =============================================================================
# The legacy reference uses a MIXED output format:
#   B1 (T), B2 (T)     -> Tesla (absolute field, post-rotation)
#   b3..b15 (units)     -> normalised (= Cn / Cm * 10000)
#   Angle (rad)         -> arg(C_m) / m  (rotation angle, pre-rotation)
#
# We run WITHOUT "nor" to keep harmonics in Tesla, then manually normalise
# n > m for comparison with the reference b_n(units).
OPTIONS = ("dri", "rot", "cel", "fed")

# Number of measurement turns per run (from Parameters.Measurement.turns)
# Flux files contain ~14 turns, but only 6 are actual measurement turns.
MEAS_TURNS_PER_RUN = 6

# Output directory for comparison reports
OUTPUT_DIR = Path("../../outputs/golden_runs/LIU_BTP8_Integral_20190717_161332")

print("Configuration loaded successfully.")
print(f"  Coil type   : Integral (A / A-B-C+D)")
print(f"  Kn file     : {KN_FILE.name}")
print(f"  Magnet order: {MAGNET_ORDER} (quadrupole)")
print(f"  R_ref       : {R_REF_M} m")
print(f"  Samples/turn: {SAMPLES_PER_TURN}")
print(f"  Options     : {OPTIONS}")
print(f"  Meas turns  : {MEAS_TURNS_PER_RUN} per run")

---
## 2. Setup and Imports

In [ ]:
import sys
import json
from datetime import datetime, timezone
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Add repo to path
repo_root = Path("../..").resolve()
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

# Core analysis imports
from rotating_coil_analyzer.analysis.kn_pipeline import (
    load_segment_kn_txt,
    compute_legacy_kn_per_turn,
    merge_coefficients,
)

print("All imports successful.")

In [ ]:
# Resolve all paths
notebook_dir = Path(".").resolve()
dataset_folder = (notebook_dir / DATASET_FOLDER).resolve()
kn_file = (notebook_dir / KN_FILE).resolve()
output_dir = (notebook_dir / OUTPUT_DIR).resolve()
output_dir.mkdir(parents=True, exist_ok=True)

print(f"Dataset folder: {dataset_folder}")
print(f"  Exists: {dataset_folder.exists()}")
print(f"Kn file: {kn_file}")
print(f"  Exists: {kn_file.exists()}")

if not dataset_folder.exists():
    raise FileNotFoundError(f"Dataset folder not found: {dataset_folder}")
if not kn_file.exists():
    raise FileNotFoundError(f"Kn file not found: {kn_file}")

---
## 3. Load Reference Results

The golden reference results were produced by the legacy C++ analyzer (ffmm framework).
Our goal is to reproduce these exact values.

In [ ]:
# Find the reference results file
ref_files = list(dataset_folder.glob("*results*.txt"))
ref_files = [f for f in ref_files if "Average" not in f.name and "Parameters" not in f.name]

if not ref_files:
    raise FileNotFoundError("No reference results file found in dataset folder")

ref_path = ref_files[0]
print(f"Loading reference: {ref_path.name}")

ref_df = pd.read_csv(ref_path, sep="\t")
print(f"\nReference data shape: {ref_df.shape}")
print(f"Number of turns: {len(ref_df)}")

# Show column names (these tell us what harmonics are available)
harmonic_cols = [c for c in ref_df.columns if c.startswith('B') or c.startswith('A')]
print(f"\nHarmonic columns: {harmonic_cols[:10]}...") 

In [ ]:
# Understand the reference data structure
print("Reference data summary:")
print(f"  Total measurement turns: {len(ref_df)}")

# Current column (may be 'I(A)' or 'I FGC(A)')
I_col = next((c for c in ref_df.columns if "I" in c and "A" in c), None)
if I_col:
    print(f"  Current column: '{I_col}'")
    print(f"  Current range: [{ref_df[I_col].min():.1f}, {ref_df[I_col].max():.1f}] A")

# Main field (B2 for quadrupole, in Tesla)
main_col = f"B{MAGNET_ORDER} (T)"
if main_col not in ref_df.columns:
    main_col = f"B{MAGNET_ORDER}(T)"
if main_col in ref_df.columns:
    print(f"  {main_col} range: [{ref_df[main_col].min():.6e}, {ref_df[main_col].max():.6e}] T")

# Detect column naming convention (Tesla vs normalized units)
has_units_cols = any("(units)" in c for c in ref_df.columns)
has_tesla_cols = any("B3 (T)" in c or "B3(T)" in c for c in ref_df.columns)
print(f"\n  Column convention: {'normalized (units)' if has_units_cols else 'Tesla'}")
print(f"  Options in data: {ref_df['Options'].iloc[0].strip() if 'Options' in ref_df.columns else 'N/A'}")

# Show first few rows
print("\nFirst 3 rows (key columns):")
key_cols = [c for c in ref_df.columns if any(
    k in c for k in ["I(A)", "I FGC", "B1", "B2", "b3", "a3", "Angle"]
)][:8]
display(ref_df[key_cols].head(3))

---
## 4. Load Kn Calibration Coefficients

The Kn coefficients encode the coil sensitivity for each harmonic order.
They depend on:
- Coil geometry (radius, turns, winding pattern)
- Compensation scheme (which coils are combined for ABS vs CMP channels)

**Important**: Use the Kn file that matches the compensation scheme in your measurement!

In [ ]:
# Load kn coefficients
kn = load_segment_kn_txt(kn_file)

print(f"Kn loaded from: {kn_file.name}")
print(f"  Harmonic orders: {list(kn.orders)}")
print(f"  Number of harmonics: {len(kn.orders)}")

# Display kn magnitudes
print("\nKn coefficient magnitudes:")
print(f"{'n':>3} {'|kn_abs|':>15} {'|kn_cmp|':>15} {'cmp/abs':>10}")
print("-" * 48)
for i, n in enumerate(kn.orders):
    abs_mag = np.abs(kn.kn_abs[i])
    cmp_mag = np.abs(kn.kn_cmp[i])
    ratio = cmp_mag / abs_mag if abs_mag > 1e-30 else np.nan
    print(f"{n:3d} {abs_mag:15.6e} {cmp_mag:15.6e} {ratio:10.4f}")

---
## 5. Parse Raw BTP8 Flux Data

### BTP8 File Format

The BTP8 flux files have 4 columns:
- Column 0: `df_abs` - Incremental flux from absolute channel (Wb)
- Column 1: `encoder` - Encoder position (counts)
- Column 2: `df_cmp` - Incremental flux from compensated channel (Wb)
- Column 3: `encoder` - Encoder position (duplicate)

Each row represents one sample (one angular position within a turn).

In [ ]:
def parse_btp8_flux_file(flux_path: Path) -> tuple:
    """Parse BTP8 flux file (4-column format).
    
    Returns:
        df_abs: Incremental flux, absolute channel (Wb)
        df_cmp: Incremental flux, compensated channel (Wb)
        encoder: Encoder counts (for timing)
    """
    data = np.loadtxt(flux_path)
    df_abs = data[:, 0]
    encoder = data[:, 1]
    df_cmp = data[:, 2]
    return df_abs, df_cmp, encoder


def parse_btp8_current_file(current_path: Path) -> np.ndarray:
    """Parse BTP8 current file (single column)."""
    return np.loadtxt(current_path)


def encoder_to_time(encoder: np.ndarray, shaft_rpm: float, 
                    encoder_res: int = 40000) -> np.ndarray:
    """Convert encoder counts to time in seconds.
    
    Args:
        encoder: Encoder counts array
        shaft_rpm: Rotation speed in RPM
        encoder_res: Encoder resolution (counts per revolution)
    """
    counts_per_second = shaft_rpm * encoder_res / 60.0
    return encoder / counts_per_second


print("Parser functions defined.")

In [ ]:
# Find all raw data files
flux_files = sorted(dataset_folder.glob("*_fluxes_Ascii.txt"))
current_files = sorted(dataset_folder.glob("*_current.txt"))

print(f"Found {len(flux_files)} flux files and {len(current_files)} current files")

if len(flux_files) == 0:
    raise FileNotFoundError("No flux files found in dataset folder")

# Show first few files
print("\nFirst 5 flux files:")
for f in flux_files[:5]:
    print(f"  {f.name}")

In [ ]:
# Inspect the first file to understand the data structure
test_flux_path = flux_files[0]
test_current_path = current_files[0]

df_abs, df_cmp, encoder = parse_btp8_flux_file(test_flux_path)
current = parse_btp8_current_file(test_current_path)
time = encoder_to_time(encoder, SHAFT_SPEED_RPM)

print(f"File: {test_flux_path.name}")
print(f"  Total samples: {len(df_abs)}")
print(f"  Samples per turn: {SAMPLES_PER_TURN}")
print(f"  Complete turns: {len(df_abs) // SAMPLES_PER_TURN}")
print(f"\nFlux ranges:")
print(f"  df_abs: [{df_abs.min():.6e}, {df_abs.max():.6e}] Wb")
print(f"  df_cmp: [{df_cmp.min():.6e}, {df_cmp.max():.6e}] Wb")

---
## 6. Process Data and Compute Harmonics

### Processing Pipeline Steps

1. **Parse** raw flux and current files
2. **Reshape** samples into turns (using SAMPLES_PER_TURN)
3. **Drift correction** (if enabled): Remove linear drift from integrated flux
4. **FFT**: Compute Fourier coefficients from flux
5. **Kn application**: Scale by coil sensitivity
6. **Rotation** (if enabled): Align phase to main field
7. **Merge**: Combine ABS and CMP channels appropriately

In [ ]:
def process_btp8_run(flux_path: Path, current_path: Path, kn,
                     samples_per_turn: int,
                     shaft_rpm: float,
                     magnet_order: int,
                     r_ref_m: float,
                     options: tuple):
    """Process a single BTP8 measurement run.
    
    This implements the same pipeline as the legacy C++ analyzer.
    
    Args:
        flux_path: Path to flux file
        current_path: Path to current file
        kn: SegmentKn calibration object
        samples_per_turn: Number of samples per revolution
        shaft_rpm: Rotation speed
        magnet_order: Main field order (m)
        r_ref_m: Reference radius for multipole normalization
        options: Tuple of enabled options ("dri", "rot", etc.)
    
    Returns:
        result: KnPerTurnResult with computed harmonics
        n_turns: Number of complete turns processed
    """
    # Parse files
    df_abs, df_cmp, encoder = parse_btp8_flux_file(flux_path)
    current = parse_btp8_current_file(current_path)
    time = encoder_to_time(encoder, shaft_rpm)
    
    # Align current to flux length (they may differ slightly)
    n_flux = len(df_abs)
    n_curr = len(current)
    if n_curr != n_flux:
        indices = np.linspace(0, n_curr - 1, n_flux).astype(int)
        current_aligned = current[indices]
    else:
        current_aligned = current
    
    # Truncate to complete turns only
    n_turns = n_flux // samples_per_turn
    n_samples = n_turns * samples_per_turn
    
    # Reshape into (n_turns, samples_per_turn)
    df_abs_turns = df_abs[:n_samples].reshape(n_turns, samples_per_turn)
    df_cmp_turns = df_cmp[:n_samples].reshape(n_turns, samples_per_turn)
    t_turns = time[:n_samples].reshape(n_turns, samples_per_turn)
    I_turns = current_aligned[:n_samples].reshape(n_turns, samples_per_turn)
    
    # Run the pipeline (this is the core computation)
    result = compute_legacy_kn_per_turn(
        df_abs_turns=df_abs_turns,
        df_cmp_turns=df_cmp_turns,
        t_turns=t_turns,
        I_turns=I_turns,
        kn=kn,
        Rref_m=r_ref_m,
        magnet_order=magnet_order,
        absCalib=1.0,
        options=options,
    )
    
    return result, n_turns


print("Processing function defined.")

In [ ]:
# Process all runs and collect results
# NOTE: Flux files contain ~14 turns each but the legacy software only uses
# MEAS_TURNS_PER_RUN (6). We compute all turns and use greedy alignment later.
print(f"Processing {len(flux_files)} measurement runs...")
print(f"Options: {OPTIONS}")
print(f"Samples per turn: {SAMPLES_PER_TURN}")
print()

all_results = []
total_turns = 0

for i, (flux_path, current_path) in enumerate(zip(flux_files, current_files)):
    result, n_turns = process_btp8_run(
        flux_path=flux_path,
        current_path=current_path,
        kn=kn,
        samples_per_turn=SAMPLES_PER_TURN,
        shaft_rpm=SHAFT_SPEED_RPM,
        magnet_order=MAGNET_ORDER,
        r_ref_m=R_REF_M,
        options=OPTIONS,
    )
    all_results.append(result)
    total_turns += n_turns
    
    if (i + 1) % 10 == 0:
        print(f"  Processed {i + 1}/{len(flux_files)} runs...")

print(f"\nDone! Processed {total_turns} total turns from {len(flux_files)} runs.")
print(f"Reference has {len(ref_df)} turns.")
print(f"(Expected: ~{MEAS_TURNS_PER_RUN} turns/run x {len(flux_files)} runs = ~{MEAS_TURNS_PER_RUN * len(flux_files)})")

---
## 7. Compare with Reference

Now we compare our computed harmonics with the golden reference values.

In [ ]:
# Build a DataFrame with all computed harmonics (in Tesla, not normalised)
# Then align turns with the reference using greedy matching on B2.
rows = []
turn_idx = 0

for result in all_results:
    C_merged, _ = merge_coefficients(
        C_abs=result.C_abs,
        C_cmp=result.C_cmp,
        magnet_order=MAGNET_ORDER,
        mode="abs_upto_m_cmp_above",
    )
    
    for t in range(C_merged.shape[0]):
        Cm = C_merged[t, MAGNET_ORDER - 1]
        Bm = Cm.real  # Main field in Tesla (real part after rotation)

        row = {"turn_idx": turn_idx}
        for i, n in enumerate(result.orders):
            C = C_merged[t, i]
            if n <= MAGNET_ORDER:
                # Store in Tesla for direct comparison with reference B_n (T)
                row[f"B{n}_T"] = C.real
                row[f"A{n}_T"] = C.imag
            else:
                # Normalise: units = C_n / B_main * 10000
                if abs(Bm) > 1e-30:
                    row[f"b{n}_units"] = C.real / Bm * 10000.0
                    row[f"a{n}_units"] = C.imag / Bm * 10000.0
                else:
                    row[f"b{n}_units"] = np.nan
                    row[f"a{n}_units"] = np.nan
        rows.append(row)
        turn_idx += 1

computed_df = pd.DataFrame(rows)
print(f"Computed results: {len(computed_df)} turns (all turns from all runs)")

# ── Greedy alignment: match each ref turn to its closest computed turn ──
# The flux files contain more turns than the reference; we find the matching
# subset by comparing the main field B2 values.
ref_b2 = ref_df["B2 (T)"].values.astype(float)
comp_b2 = computed_df["B2_T"].values

aligned_indices = []
used = set()

for ri in range(len(ref_b2)):
    best_ci = -1
    best_diff = 1e30
    for ci in range(len(comp_b2)):
        if ci in used:
            continue
        d = abs(comp_b2[ci] - ref_b2[ri])
        if d < best_diff:
            best_diff = d
            best_ci = ci
    rel = best_diff / max(abs(ref_b2[ri]), 1e-30)
    if rel < 1e-6:  # accept matches better than 1 ppm
        aligned_indices.append(best_ci)
        used.add(best_ci)

aligned_df = computed_df.iloc[aligned_indices].reset_index(drop=True)
print(f"Aligned turns: {len(aligned_df)} / {len(ref_df)} reference turns matched")
if len(aligned_df) < len(ref_df):
    print(f"  WARNING: {len(ref_df) - len(aligned_df)} reference turns could not be matched!")

In [ ]:
# Compare harmonics using aligned turns
n_compare = len(aligned_df)
print(f"Comparing {n_compare} aligned turns...\n")

def _find_ref_col(n, component="B"):
    """Find the reference column for harmonic n, normal (B/b) or skew (A/a)."""
    for pattern in [
        f"{component}{n} (T)",   f"{component}{n}(T)",
        f"{component.lower()}{n} (units)", f"{component.lower()}{n}(units)",
        f"{component}{n} (units)", f"{component}{n}(units)",
        f"{component}{n}",
    ]:
        if pattern in ref_df.columns:
            return pattern
    return None

print("=" * 98)
print("HARMONIC COMPARISON SUMMARY")
print("=" * 98)
print(f"{'n':>3} {'Ref col':>20} {'Comp col':>16} {'Max |Diff|':>14} {'Max |Rel|':>14} {'RMS':>14} {'Status':>12}")
print("-" * 98)

comparison_results = []

for n in range(1, 16):
    ref_col = _find_ref_col(n, "B")
    if ref_col is None:
        continue

    # Pick the matching computed column based on format
    if n <= MAGNET_ORDER:
        comp_col = f"B{n}_T"     # Tesla
    else:
        comp_col = f"b{n}_units" # normalised

    if comp_col not in aligned_df.columns:
        continue

    comp_vals = aligned_df[comp_col].values[:n_compare]
    ref_vals = ref_df[ref_col].values[:n_compare].astype(float)

    abs_diff = np.abs(comp_vals - ref_vals)
    max_abs_diff = np.max(abs_diff)
    rms_diff = np.sqrt(np.mean(abs_diff**2))

    with np.errstate(divide="ignore", invalid="ignore"):
        rel_diff = np.where(
            np.abs(ref_vals) > 1e-30,
            np.abs((comp_vals - ref_vals) / ref_vals),
            0.0,
        )
    max_rel_diff = float(np.nanmax(rel_diff))

    if max_rel_diff < 1e-6:     status = "EXCELLENT"
    elif max_rel_diff < 1e-3:   status = "GOOD"
    elif max_rel_diff < 0.1:    status = "CLOSE"
    elif max_rel_diff < 1.0:    status = "MARGINAL"
    else:                        status = "MISMATCH"

    print(f"{n:3d} {ref_col:>20s} {comp_col:>16s} {max_abs_diff:14.6e} {max_rel_diff:14.6e} {rms_diff:14.6e} {status:>12}")

    comparison_results.append({
        "n": n,
        "ref_col": ref_col,
        "comp_col": comp_col,
        "max_abs_diff": max_abs_diff,
        "max_rel_diff": max_rel_diff,
        "rms_diff": rms_diff,
        "status": status,
    })

In [ ]:
# Detailed comparison for main field (B2 for quadrupole)
main_n = MAGNET_ORDER
comp_col = f"B{main_n}_T"
ref_col = _find_ref_col(main_n, "B")

if ref_col is None:
    print(f"Reference column for n={main_n} not found -- skipping detailed comparison.")
else:
    print(f"\nDetailed comparison for main field (n={main_n}), ref col = '{ref_col}':")
    print(f"{'Turn':>6} {'Computed':>16} {'Reference':>16} {'Diff':>16} {'Rel Diff':>12}")
    print("-" * 70)

    for i in range(min(10, n_compare)):
        comp_val = aligned_df[comp_col].iloc[i]
        ref_val = float(ref_df[ref_col].iloc[i])
        diff = comp_val - ref_val
        rel_diff = diff / ref_val if abs(ref_val) > 1e-20 else 0
        print(f"{i:6d} {comp_val:16.9e} {ref_val:16.9e} {diff:16.9e} {rel_diff:12.6e}")

---
## 8. Visualization

In [ ]:
# Plot computed vs reference for main field
ref_main_col = _find_ref_col(MAGNET_ORDER, "B")

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# B2 time series
ax = axes[0, 0]
ref_vals = ref_df[ref_main_col].values[:n_compare].astype(float)
comp_vals = aligned_df[f"B{MAGNET_ORDER}_T"].values[:n_compare]
ax.plot(ref_vals, 'b-', label='Reference', alpha=0.7)
ax.plot(comp_vals, 'r--', label='Computed', alpha=0.7)
ax.set_xlabel('Turn')
ax.set_ylabel(ref_main_col)
ax.set_title(f'Main Field (n={MAGNET_ORDER}): Time Series')
ax.legend()
ax.grid(True, alpha=0.3)

# B2 scatter (computed vs reference)
ax = axes[0, 1]
ax.scatter(ref_vals, comp_vals, alpha=0.5, s=10)
lims = [min(ref_vals.min(), comp_vals.min()), max(ref_vals.max(), comp_vals.max())]
ax.plot(lims, lims, 'k--', label='Perfect match')
ax.set_xlabel('Reference')
ax.set_ylabel('Computed')
ax.set_title(f'B{MAGNET_ORDER}: Computed vs Reference')
ax.legend()
ax.grid(True, alpha=0.3)

# Difference histogram
ax = axes[1, 0]
diff = comp_vals - ref_vals
ax.hist(diff, bins=50, edgecolor='black', alpha=0.7)
ax.axvline(0, color='r', linestyle='--', label='Zero')
ax.set_xlabel('Difference (T)')
ax.set_ylabel('Count')
ax.set_title(f'B{MAGNET_ORDER}: Difference Distribution')
ax.legend()
ax.grid(True, alpha=0.3)

# Relative difference by harmonic
ax = axes[1, 1]
harmonics = [r['n'] for r in comparison_results]
rel_diffs = [r['max_rel_diff'] for r in comparison_results]
colors = ['green' if d < 1e-3 else 'orange' if d < 0.1 else 'red' for d in rel_diffs]
ax.bar(harmonics, rel_diffs, color=colors, edgecolor='black')
ax.axhline(1e-3, color='g', linestyle='--', label='Good threshold')
ax.axhline(0.1, color='orange', linestyle='--', label='Close threshold')
ax.set_xlabel('Harmonic Order n')
ax.set_ylabel('Max Relative Difference')
ax.set_title('Parity Check by Harmonic')
ax.set_yscale('log')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'parity_comparison.png', dpi=150)
plt.show()

print(f"\nPlot saved to: {output_dir / 'parity_comparison.png'}")

---
## 9. How to Replicate Using the GUI

### Step 1: Launch the GUI

```bash
py -m rotating_coil_analyzer.gui.app
```

### Step 2: File Browser Tab

1. Navigate to `golden_standards/golden_standard_01_LIU_BTP8/Integral/20190717_161332_LIU`
2. Select the flux and current files

### Step 3: Coil Calibration Tab

1. **Load Kn File**: Select the R45_PCB_N1 A_ABCD file:
   `COIL_PCB/Kn_R45_PCB_N1_0001_A_ABCD.txt`
   - Absolute channel: A (single coil)
   - Compensated channel: A-B-C+D (long integral bucking)

### Step 4: Harmonic Merge Tab

1. **Magnet Order**: 2 (quadrupole)
2. **Reference Radius**: 0.059 m
3. **Merge Mode**: "ABS up to m, CMP above"
4. **Compensation Scheme**: A / A-B-C+D

### Step 5: Analysis Options

1. Enable **all** options: Drift (dri), Rotation (rot), Normalization (nor), Center Location (cel), Feeddown (fed)
2. **Samples Per Turn**: 512 (BTP8 format)
3. Run and export

### Verification

Compare exported CSV with the reference `BTP8_20190717_161332_results.txt`:
- B2 (T) should match to within < 1 ppm relative error
- Normalized harmonics b3..b6 (units) should match to ~1-3%
- Higher-order harmonics (b7+) show larger relative errors due to feeddown amplification

### Notes on Turn Alignment

The flux files contain ~14 turns per run, but the legacy software uses only 6
(set by `Parameters.Measurement.turns`). The notebook handles this via greedy
matching on the main field B2. When comparing directly, ensure you are comparing
the same physical measurement turns.

---
## 10. Export Results

In [ ]:
# Export aligned computed results
computed_path = output_dir / "computed_results_aligned.csv"
aligned_df.to_csv(computed_path, index=False)
print(f"Aligned computed results saved to: {computed_path}")

# Export comparison summary
comparison_df = pd.DataFrame(comparison_results)
comparison_path = output_dir / "comparison_summary.csv"
comparison_df.to_csv(comparison_path, index=False)
print(f"Comparison summary saved to: {comparison_path}")

# Export provenance metadata
metadata = {
    "timestamp": datetime.now(timezone.utc).isoformat(),
    "dataset_folder": str(dataset_folder),
    "kn_file": str(kn_file),
    "magnet_order": MAGNET_ORDER,
    "r_ref_m": R_REF_M,
    "samples_per_turn": SAMPLES_PER_TURN,
    "options": OPTIONS,
    "n_computed_turns": len(computed_df),
    "n_aligned_turns": len(aligned_df),
    "n_reference_turns": len(ref_df),
}

metadata_path = output_dir / "provenance.json"
with open(metadata_path, "w") as f:
    json.dump(metadata, f, indent=2)
print(f"Provenance metadata saved to: {metadata_path}")

---
## 11. Final Summary

In [ ]:
# Print final summary
print("=" * 80)
print("GOLDEN STANDARD PARITY VALIDATION - FINAL REPORT")
print("=" * 80)
print(f"\nGenerated: {datetime.now().isoformat()}")
print(f"\nDataset: {dataset_folder.name}")
print(f"Kn file: {kn_file.name}")
print(f"\nConfiguration:")
print(f"  Magnet order: {MAGNET_ORDER}")
print(f"  Reference radius: {R_REF_M} m")
print(f"  Samples per turn: {SAMPLES_PER_TURN}")
print(f"  Options: {OPTIONS}")

print(f"\nData processed:")
print(f"  Computed turns: {len(computed_df)} (all turns from flux files)")
print(f"  Aligned turns: {len(aligned_df)} (matched to reference)")
print(f"  Reference turns: {len(ref_df)}")

print(f"\nParity Results:")
excellent = sum(1 for r in comparison_results if r['status'] == 'EXCELLENT')
good = sum(1 for r in comparison_results if r['status'] == 'GOOD')
close = sum(1 for r in comparison_results if r['status'] == 'CLOSE')
marginal = sum(1 for r in comparison_results if r['status'] == 'MARGINAL')
mismatch = sum(1 for r in comparison_results if r['status'] == 'MISMATCH')

print(f"  EXCELLENT (< 1e-6 rel): {excellent} harmonics")
print(f"  GOOD (< 1e-3 rel):      {good} harmonics")
print(f"  CLOSE (< 0.1 rel):      {close} harmonics")
print(f"  MARGINAL (< 1.0 rel):   {marginal} harmonics")
print(f"  MISMATCH (>= 1.0 rel):  {mismatch} harmonics")

if mismatch == 0 and marginal == 0:
    print("\n" + "*" * 80)
    print("VALIDATION PASSED: All harmonics match within acceptable tolerance!")
    print("*" * 80)
elif mismatch == 0:
    print("\n" + "*" * 80)
    print("VALIDATION MOSTLY PASSED: Main field excellent, some marginal harmonics.")
    print("*" * 80)
else:
    print("\n" + "!" * 80)
    print(f"VALIDATION PARTIAL: Main field correct, {mismatch} higher harmonics diverge.")
    print("Higher-order errors are expected due to feeddown amplification and CMP noise.")
    print("!" * 80)

print(f"\nOutput files:")
print(f"  {computed_path}")
print(f"  {comparison_path}")
print(f"  {metadata_path}")
print(f"  {output_dir / 'parity_comparison.png'}")